## Welcome to Week 3 (what you've all been waiting for), Single Cell RNA!

### In this week, we're going to focus on the basics of single cell RNA analysis, including dimensionality reduction and opitional doublet detection and cell type identification/cell surface protein integration.

<u>First</u>, we'll have an introduction to the main R package used for single cell analysis - **Seurat**, developed by Rajul Satija's lab at the New York Genome Center. There are a few publications on how Seurat works that you can check out if you want more detail about what's going on under the hood:
* https://www.ncbi.nlm.nih.gov/pubmed/29608179
* https://www.ncbi.nlm.nih.gov/pubmed/31178118

Seurat starts from the output of the 10X Genomics cellranger pipeline, which is used for processing of single cell RNA data generated using droplet-based capture. You can read more about how single cell RNA sequencing and the cellranger pipeline work here (and we may do a cellranger tutorial later if there is interest:  
* https://www.10xgenomics.com/products/single-cell/?src=search&lss=google&cnm=sem-goog-2020-website-page-ra_g-p_brand-amr-retarget&cid=7011P000000oWys (videos at the end)
* https://support.10xgenomics.com/single-cell-gene-expression/software/overview/welcome  

The main idea is that single cells are captured in oil-water emulsion droplets along with barcoded oligos, which tag RNA transcripts with a unique cellular barcode during reverse transcription. After sequencing, the cellranger pipeline aligns the transcripts and matches them back to the cellular barcode to obtain counts for each transcript that was expressed for each cell. The main output that we will use from the cellranger pipeline is the **feature/cell matrix**, which is a matrix of cells by genes, where the values are the number of transcripts that were detected for that cell.  

Single cell methods are not perfect, and one of the main things we have to deal with is **dropout**, which occurs when a transcript is expressed by a cell but is not recovered by single cell sequencing. Lowly expressed genes are more likely to be affected by dropout. For 10X datasets, typically we can expect to detect 1000-2000 genes per cell. The main approach that has been developed to deal with dropout during single cell analysis is dimensionality reduction. The main steps we will take in Seurat are:
1. **Filter** cells based on QC metrics (number of genes detected, percent mitochondrial genes, etc.)
2. **Normalize** data
3. Identify **variable** genes to use for principal component analysis (PCA)
4. **Scale** data and regress out unwanted sources of variation (i.e. cell cycle)
5. **PCA** dimensionality reduction
6. **Cluster** cells to find nearest neighbors in PCA space (typically corresponding to different cell types/states)
7. Visualize cells using **UMAP/TSNE** non-linear dimensionality reduction
8. Find **differentially expressed** genes between clusters
9. **Cluster annotation**  

There is obviously a lot we can do after this but these steps are common to any single cell project. To get started we will be using this publically available human PBMC dataset from 10X:
* https://support.10xgenomics.com/single-cell-vdj/datasets/3.0.0/vdj_v1_hs_pbmc2_5gex_protein  

Go ahead and download the **Feature / cell matrix (filtered)**. Unpack the tar.gz file either using Finder or with this command in terminal:  
`tar -xzf vdj_v1_hs_pbmc2_5gex_protein_filtered_feature_bc_matrix.tar.gz`  

You can also take a look at the **Summary HTML** which has the QC metrics from the cellranger pipeline. We will now be following the Seurat Guided Clustering Tutorial found here (but using the larger PBMC dataset we just downloaded):
* https://satijalab.org/seurat/v3.1/pbmc3k_tutorial.html

## Important install note:
Prior to installing Seurat from cran, run the following commands in terminal in your base conda environment:  
`conda install -c conda-forge r-igraph r-leiden r-rcurl r-hdf5r`  
`conda install -c bioconda bioconductor-rhdf5 bioconductor-biocparallel`

Then restart this kernel.

In [ ]:
# First let's install Seurat which is available from cran. This may take a little while.
install.packages('BiocManager')
BiocManager::install('multtest')
install.packages('Seurat')

In [ ]:
# Load required packges:
library(dplyr)
library(Seurat)
library(patchwork)

In [ ]:
# Now we will follow the Seurat tutorial linked above:

# Load the PBMC dataset - replace with the path to the filtered_feature_bc_matrix folder you unpacked
pbmc.data <- Read10X(data.dir = "filtered_feature_bc_matrix")

In [ ]:
# Note that this dataset includes both gene and antibody information.
# To initialize the Seurat object with just the RNA data, we modify this line as follows.

# Initialize the Seurat object with the raw (non-normalized data).
pbmc <- CreateSeuratObject(counts = pbmc.data$`Gene Expression`, project = "pbmc", min.cells = 3
                           , min.features = 200)
pbmc

### The rest of the tutorial I will leave for you to fill in!  
There are a couple of important pivot points that you can modify for your data which I will break up by processing step:
1. **Filter**  
   a. `nFeature_RNA`: How many genes are detected per cell. I typically use 200 as a minimum but check the distribution in your data. Cells with fewer than 200 genes detected typically don't have enough information to cluster on.  
   b. `nCount_RNA`: How many UMIs or transcripts detected per cell. I typically don't filter using this metric as it is linearly correlated with nFeature_RNA. Some people will use an upper threshold here to try and filter out doublets but typically is not robust and we describe alternative approaches below.  
   c. `percent.mt`: Percent mitochondrial transcripts. Dying cells will have more mitochondrial gene content and so this can be used to filter low quality cells. Check the distribution in your data but typically upper thresholds of 5-10% mitochondrial content make sense.
2. **Normalize** - I use the defaults pretty much always.
3. **Variable genes** - Not a lot to modify here. If you have residual batch/unwanted variation (like cell cycle) effects after regression you can remove these genes from the variable gene set. Sometimes I also observed clustering based on variable immune genes (like TCR/Ig transcripts) so these can be removed as well.
4. **Scale**  
   a. `vars.to.regress`: If you have batch effects/sources of unwanted variation like cell cycle effects you can regress them out here. Some other common variables to regress are nCount_RNA, percent.mt (if you observe clustering based on these features & not due to remaining low quality cells).
5. **PCA**  
   a. `dims`: Pick how many dimensions to use for PCA. I typically use ElbowPlot to decide. Including more PCs is usually better than too few, so I would err past the elbow. Between 10-40 PCs is typically appropriate.
6. **Cluster**  
   a. `resolution`: Argument to the FindClusters function. This is a big one you can change. Higher resolution values will give more clusters, lower resolution values give fewer clusters. How many clusters you want depends on the biological variation in your data as well as how you want to analyze it (are broad cell types enough or do you want specific cell states?). Values between 0.4-1.2 are typically appropriate. 
7. **UMAP** - UMAP is superior to TSNE in many ways so I won't talk about TSNE.  
   a. `min.dist`: This will change how spread out the UMAP is with lower values pulling cells apart more. This is useful for closely related cells that may not seperate well in the UMAP using the default parameters.

## Additional steps
Congrats on finishing the Seurat tutorial! There are a few things you can do now to explore the data:
1. Check `nFeature_RNA` and `percent.mt` for each cluster using the `VlnPlot` function. Do any clusters look like mostly low quality cells?
2. Check for cell cycle effects by checking expression of cell cycle associated genes using the `VlnPlot` function or looking for cell cycle genes in the cluster markers found by `FindAllMarkers`. If cell cycle effects are present they can be regressed out using this vignette:  
https://satijalab.org/seurat/v3.1/cell_cycle_vignette.html
3. Annotate cell clusters based on expression of marker genes. You can check expression using either the `VlnPlot` or `FeaturePlot` functions. Both are useful! Here are couple of good markers for PBMCs:  
   a. T cells: CD3G, CD3E, CD3D  
   b. Cytotoxic CD8+ T cells: CD8A  
   c. Helper CD4+ T cells: CD4  
   d. Naive T cells: CCR7  
   e. Memory CD8+ cells: EOMES  
   f. Effector memory CD8+ T cells: KLRD1  
   g. B cells: CD19, MS4A1  
   h. Monocytes: CD14, FCGR3A  
   i. NK cells: GNLY, NKG7 (CD3-, NK cells and cytotoxic T cells share many markers and may cluster together)  
   j. Platelets: PPBP
4. Subset a broad cell type and recluster to find more accurate subtypes. For example, subset all T cell clusters using the `subset` function and recluster starting from data scaling and variable gene detection. Do you find more clusters? How do they compare to the original clusters you found using all PBMCs? Were there any non-T cells that previosly clustered together with T cells?
5. Make sure you save your Seurat object (`pbmc`) so it can be loaded again. Also try saving some plots as PDFs and tables (like the cluster markers from `FindAllMarkers`) as text files.

**Extra challenge**: This dataset also has CITE-seq surface protein data! Try adding this data to your Seurat object (Hint look back at the `pbmc.data` we read in at the beginning). Which proteins were analyzed? How does protein expression compare to RNA expression? See more details in the vignette here:  
https://satijalab.org/seurat/v3.1/multimodal_vignette.html

## Optional: Perform double detection using DoubletFinder
One confounding aspect of single cell data is the precense of cell doublets, where 2 cells can be captured in the same droplet, thus receiving the same cell barcode for sequencing and introducing artifacts into the data. The doublet rate for droplet based methods scales linearly with the number of cells captured, so for our dataset of \~9,000 cells we can expect a doublet rate of **\~7% or \~600 cells**. Several studies have compared features that seperate true cell doublets (based on SNP or other information) and typically cell doublets do not have easily seperable features such as number of genes detected, etc. More sophisticated approches using simulated cell mixtures have better success. One such package is DoubletFinder, and you can read the details here:
* https://www.ncbi.nlm.nih.gov/pubmed/30954475  

Some nice features of doublet finder is that it is directly compatible with Seurat. Let's give it a try on our data using the documentation found here:
* https://github.com/chris-mcginnis-ucsf/DoubletFinder

**Note some changes made in Seurat v3:**  
`seu_kidney@cell.names` would now be `Cells(seu_kidney)`  

Also depending on which parameters you choose, you may have to change the value of the `reuse.pANN` variable from  
`reuse.pANN = "pANN_0.25_0.09_913"` to another string. Check the metadata (`head(pbmc@meta.data)`) after running the first line.

### Follow-up
How many doublets did we find? (hint: try using `table` on the DF column in `pbmc@meta.data`) How does the number of genes detected compare for singlets vs doublets? Which clusters are doublets enriched in? Where are doublets located on the UMAP? 

Remove predicted doublets from your data and rerun clustering and UMAP analysis. Does it look better?

## Optional: Automated cell type annotation using SingleR
Manual cell type annotation can take time if you are not sure what good markers are for your expected cell types. There are a few packages that can be used for automated cell type annotation using annotated references. One such package is **SingleR** which we will be using. Some notes of caution - automated cell type annotation is only as good as the reference. If you try to annotate neural cell types with an immune references the analysis will not very useful. Proceed with caution and always think about your results/confirm with multiple methods. Check out the publication and documentation for SingleR here:
* https://www.ncbi.nlm.nih.gov/pubmed/30643263
* https://bioconductor.org/packages/release/bioc/html/SingleR.html

### Follow-up
How does SingleR's annotation compare to your manual cell type annotation? How do the results different references compare? You can check available references here: https://bioconductor.org/packages/release/bioc/vignettes/SingleR/inst/doc/SingleR.html#5_available_references

Congrats on finishing your first single cell analysis! Our next workshop will include more custom analysis and plotting as well as integration of TCR/BCR data.